This code block handles the import statements and data loading, to keep the notebook clean.

In [1]:
import pandas as pd
import kagglehub

#Philadelphia 2025 Crime Statistics, from our github
url = "https://raw.githubusercontent.com/pwax1120/mcdermid_waxman_final_project/main/data/incidents_part1_part2.csv"
phil_test = pd.read_csv(url)

#this downloads the kaggle nfl dataset
path_nfl = kagglehub.dataset_download("keonim/nfl-game-scores-dataset-2017-2023")
game_dates = pd.read_csv(f"{path_nfl}/Season_Scores/2024_scores.csv")


In [2]:
phil_test

,the_geom,cartodb_id,the_geom_webmercator,objectid,dc_dist,psa,dispatch_date_time,dispatch_date,dispatch_time,hour,dc_key,location_block,ucr_general,text_general_code,point_x,point_y,lat,lng
0,0101000020E610000028DE23F684CB52C0F80A326C8EF5...,1,0101000020110F0000A2A1698FD9EC5FC1F0CB69305384...,27742612,1,1,2025-04-05 08:57:00+00,2025-04-05,04:57:00,4,2.025010e+11,1900 BLOCK OREGON AV,300,Robbery No Firearm,-75.179990,39.918409,39.918409,-75.179990
1,0101000020E6100000D0DAEDE02BCB52C0B8DD2D5E66F6...,2,0101000020110F0000F586773E42EC5FC1F34D75544285...,27742613,1,1,2025-04-15 06:38:00+00,2025-04-15,02:38:00,2,2.025010e+11,2000 BLOCK S 17th St,300,Robbery No Firearm,-75.174553,39.924999,39.924999,-75.174553
2,0101000020E6100000D4B4B5329FC552C008A74C71A205...,3,0101000020110F0000BBC23C13D5E25FC1E6080B4C2596...,27742614,2,2,2025-04-28 14:13:00+00,2025-04-28,10:13:00,10,2.025020e+11,6300 BLOCK OXFORD AV,300,Robbery No Firearm,-75.087842,40.044020,40.044020,-75.087842
3,0101000020E6100000A0B2614D65C652C030732CEFAA03...,4,0101000020110F0000DCF7359325E45FC12842C9C4F693...,27742615,2,1,2025-04-29 17:08:00+00,2025-04-29,13:08:00,13,2.025020e+11,4600 BLOCK E ROOSEVELT BLVD,300,Robbery No Firearm,-75.099933,40.028654,40.028654,-75.099933
4,0101000020E610000070D5E44540CA52C0F82BAEA803F6...,5,0101000020110F00000F654F0BB2EA5FC1B9E8ED03D584...,27742616,3,3,2025-03-26 21:32:00+00,2025-03-26,17:32:00,17,2.025030e+11,2100 BLOCK S 8TH ST,300,Robbery No Firearm,-75.160173,39.921987,39.921987,-75.160173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55319,0101000020E6100000DB3E9AA03CC952C02FD516E824FF...,3391472,0101000020110F000073B18702F9E85FC1CDAB3AADF28E...,24568828,25,4,2025-01-10 16:51:00+00,2025-01-10,11:51:00,11,2.025250e+11,700 BLOCK W LEHIGH AV,400,Aggravated Assault Firearm,-75.144325,39.993314,39.993314,-75.144325
55320,0101000020E61000000040A95123C952C034D7ABC07701...,3391483,0101000020110F00002C696505CEE85FC19DC901328691...,24687788,25,1,2025-01-21 11:07:00+00,2025-01-21,06:07:00,6,2.025250e+11,3900 BLOCK N PERCY ST,300,Robbery Firearm,-75.142781,40.011467,40.011467,-75.142781
55321,0101000020E610000005FCB71E87CA52C0FA7EACD846FB...,3391487,0101000020110F00003C3B88622AEB5FC1EB73315DA98A...,25432284,9,1,2025-02-15 01:32:00+00,2025-02-14,20:32:00,20,2.025090e+11,1500 BLOCK SPRING GARDEN ST,400,Aggravated Assault No Firearm,-75.164497,39.963100,39.963100,-75.164497
55322,0101000020E6100000C1D2F17F64C852C0A4946B651D04...,3391491,0101000020110F0000CDA93CE589E75FC19E8A28BA7594...,25521124,35,2,2025-02-11 18:56:00+00,2025-02-11,13:56:00,13,2.025350e+11,5300 BLOCK N 5TH ST,300,Robbery Firearm,-75.131134,40.032147,40.032147,-75.131134


In [3]:
#game_date takes the team and year, and returns data_frame
def nfl_game_date(year, team):
    #load the specific year
    df = pd.read_csv(f"{path_nfl}/Season_Scores/{year}_scores.csv")
    df = df.dropna(subset=['Date'])

    
    #subsets the df to games where the team is playing
    df_away = df[(df["AwayTeam"] == team)]
    df_home = df[(df["HomeTeam"] == team)]
        
    home_dates_with_year = df_home['Date'].apply(
        lambda x: f"{x}/{year+1}" if int(str(x).split('/')[0]) <= 6 
        else f"{x}/{year}")    
    away_dates_with_year = df_away['Date'].apply(
        lambda x: f"{x}/{year+1}" if int(str(x).split('/')[0]) <= 6 
        else f"{x}/{year}")
    
    # Now convert to datetime
    home_game_dates = pd.to_datetime(home_dates_with_year)
    away_game_dates = pd.to_datetime(away_dates_with_year)

    #Gets the result of the game
    home_game_win = df_home['HomeWin'].to_list()
    away_game_win = df_away['AwayWin'].to_list()

    #Convert to boolean
    home_game_win = [bool(x) for x in home_game_win]
    away_game_win = [bool(x) for x in away_game_win]

    #Creates an away, home df, with the pairs for results and tags the Location
    home = pd.DataFrame(list(zip(home_game_dates, home_game_win)))
    away = pd.DataFrame(list(zip(away_game_dates, away_game_win)))
    home["Location"] = "Home"
    away["Location"] = "Away"

    #puts the two together and sorts them by date
    season = pd.concat([home, away], ignore_index=True) 
    season.columns = ["Date", "Result", "Location"]
    season = season.sort_values('Date').reset_index(drop=True)    
    
    return season

In [4]:
nfl_game_date(2021, "Eagles")

,Date,Result,Location
0,2021-08-12,False,Home
1,2021-08-19,False,Home
2,2021-08-27,False,Away
3,2021-09-12,True,Away
4,2021-09-19,False,Home
5,2021-09-27,False,Away
6,2021-10-03,False,Home
7,2021-10-10,True,Away
8,2021-10-14,False,Home
9,2021-10-24,False,Away


In [5]:
#this downloads the kaggle nba dataset
path_nba = kagglehub.dataset_download("eoinamoore/historical-nba-data-and-player-box-scores")

100%|██████████████████████████████████████| 93.0M/93.0M [00:06<00:00, 15.5MB/s]

Extracting files...


In [6]:
def nba_game_date(team,year):
    #first get teamId for this dataset
    team_df = pd.read_csv(f"{path_nba}/TeamHistories.csv")

    #searches the dataset for the ID, returns nothing if not found
    team_id = team_df[team_df['teamName'] == team]['teamId']
    if not team_id.empty:
        team_id = team_id.iloc[0]
    else:
        print("Team not found in dataset, look at the documentation.")
        return

    #use team_id and year to sort games wanted
    df = pd.read_csv(f"{path_nba}/Games.csv", low_memory=False)
    df["gameDate"] = pd.to_datetime(df["gameDate"])
    start = pd.to_datetime(f"{year}-10-01")
    end = pd.to_datetime(f"{year+1}-06-30")
    df_home = df[(df["hometeamId"] == team_id) & ((df['gameDate'] >= start) & (df['gameDate'] <= end))]
    df_away = df[(df["awayteamId"] == team_id) & ((df['gameDate'] >= start) & (df['gameDate'] <= end))]


    home_game_dates = df_home["gameDate"].to_list()
    away_game_dates = df_away["gameDate"].to_list()
    
    home_game_win = [x == team_id for x in df_home['winner']]
    away_game_win = [x == team_id for x in df_away['winner']]

    #Creates an away, home df, with the pairs for results and tags the Location
    home = pd.DataFrame(list(zip(home_game_dates, home_game_win)))
    away = pd.DataFrame(list(zip(away_game_dates, away_game_win)))
    home["Location"] = "Home"
    away["Location"] = "Away"
    season = pd.concat([home, away], ignore_index=True) 
    season.columns = ["Date", "Result", "Location"]
    season = season.sort_values('Date').reset_index(drop=True)    
    
    return season

In [7]:
nba_game_date("76ers",2015)

,Date,Result,Location
0,2015-10-06 19:00:00,False,Away
1,2015-10-08 19:00:00,True,Home
2,2015-10-10 15:00:00,True,Home
3,2015-10-12 19:30:00,False,Away
4,2015-10-16 19:00:00,False,Home
...,...,...,...
84,2016-04-05 19:00:00,True,Home
85,2016-04-08 19:00:00,False,Home
86,2016-04-10 17:00:00,False,Home
87,2016-04-12 19:30:00,False,Away
